In [50]:
import re
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANACLARASAMARINO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
df_todas_categorias = pd.read_sql("""
SELECT DISTINCT 
	categoria_atual
FROM
	anuncios_resumo
""", con)

In [40]:
lista_todas_categorias = [*df_todas_categorias['categoria_atual']]

In [41]:
def monta_sql_por_categoria(categoria):
    return f"""
    SELECT DISTINCT 
	url_lista,
	numero_pagina_lista,
	url_anuncio,
	categoria_completa,
	categoria_atual,
	titulo_anuncio,
	detalhes_anuncio,
	preco_anuncio,
	url_img_principal,
	qt_img_anuncio,
	localizacao_completa,
	localizacao_complemento,
	anuncio_profissional,
	data_publicacao_anuncio,
	data_coleta_dados
FROM
	anuncios_resumo
WHERE categoria_atual = '{categoria}'
    """

In [42]:
monta_sql_por_categoria('Venda - casas e apartamentos')
pd.read_sql(monta_sql_por_categoria(lista_todas_categorias[0]),con)

,url_lista,numero_pagina_lista,url_anuncio,categoria_completa,categoria_atual,titulo_anuncio,detalhes_anuncio,preco_anuncio,url_img_principal,qt_img_anuncio,localizacao_completa,localizacao_complemento,anuncio_profissional,data_publicacao_anuncio,data_coleta_dados
0,https://ac.olx.com.br/acre/rio-branco/imoveis/...,1,https://ac.olx.com.br/acre/imoveis/vendo-hevea...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,Vendo Hevea 03 quartos,3 quartos | 80m² | Condomínio: R$ 0 | 2 ...,460000.0,https://img.olx.com.br/thumbs256x256/23/235264...,3,Brasil;AC;DDD 68 - Acre;Rio Branco,"Rio Branco, Nova Esperança",1,2022-09-17 16:55,2022-09-17 23:57
1,https://ac.olx.com.br/acre/rio-branco/imoveis/...,1,https://ac.olx.com.br/acre/imoveis/vendo-apart...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,VENDO APARTAMENTO NO LÁ RESERVE,3 quartos | 78m² | 2 vagas |,500000.0,https://img.olx.com.br/thumbs256x256/52/528217...,11,Brasil;AC;DDD 68 - Acre;Rio Branco,"Rio Branco, Residencial Petrópolis",0,2022-09-17 16:24,2022-09-17 23:57
2,https://ac.olx.com.br/acre/rio-branco/imoveis/...,1,https://ac.olx.com.br/acre/imoveis/apartamento...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,Apartamento Disponível,2 quartos | 55m² | Condomínio: R$ 200 | ...,115000.0,https://img.olx.com.br/thumbs256x256/52/529202...,10,Brasil;AC;DDD 68 - Acre;Rio Branco,"Rio Branco, Estação Experimental",0,2022-09-17 15:17,2022-09-17 23:57
3,https://ac.olx.com.br/acre/rio-branco/imoveis/...,1,https://ac.olx.com.br/acre/imoveis/vendo-apto-...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,Vendo Apto Manoel Julião,2 quartos | 49m² | Condomínio: R$ 0 | 1 ...,120000.0,https://img.olx.com.br/thumbs256x256/23/238207...,9,Brasil;AC;DDD 68 - Acre;Rio Branco,"Rio Branco, Conjunto Manoel Julião",1,2022-09-17 13:59,2022-09-17 23:57
4,https://ac.olx.com.br/acre/rio-branco/imoveis/...,1,https://ac.olx.com.br/acre/imoveis/apartamento...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,Apartamento na estrada do Quixadá km 02,1 quarto |,25000.0,https://img.olx.com.br/thumbs256x256/52/528210...,1,Brasil;AC;DDD 68 - Acre;Rio Branco,"Rio Branco, Vitória",0,2022-09-17 00:23,2022-09-17 23:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140938,https://to.olx.com.br/tocantins/outras-cidades...,10,https://to.olx.com.br/tocantins/imoveis/sala-c...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,Sala comercial em Luzimangues em fase de Const...,1 quarto | 300m² | Condomínio: R$ 0 |,170000.0,https://img.olx.com.br/thumbs256x256/80/802292...,5,Brasil;TO;DDD 63 - Tocantins;Outras Cidades,Porto Nacional,1,2022-07-07 10:36,2022-09-18 01:58
1140939,https://to.olx.com.br/tocantins/outras-cidades...,10,https://to.olx.com.br/tocantins/imoveis/casa-p...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,CASA PORTO NACIONAL,5 ou mais quartos | 200m² | Condomínio: R$...,400000.0,https://img.olx.com.br/thumbs256x256/74/744242...,16,Brasil;TO;DDD 63 - Tocantins;Outras Cidades,Porto Nacional,1,2022-07-01 13:57,2022-09-18 01:58
1140940,https://to.olx.com.br/tocantins/outras-cidades...,10,https://to.olx.com.br/tocantins/imoveis/casa-a...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,"Casa à venda no bairro Fazenda Velha, em Cajamar",5 ou mais quartos | 484m² | Condomínio: R$...,2350000.0,https://img.olx.com.br/thumbs256x256/52/520256...,20,Brasil;TO;DDD 63 - Tocantins;Outras Cidades,Porto Nacional,1,2022-06-10 04:39,2022-09-18 01:58
1140941,https://to.olx.com.br/tocantins/outras-cidades...,10,https://to.olx.com.br/tocantins/imoveis/casa-a...,Imóveis;Venda - casas e apartamentos,Venda - casas e apartamentos,"Casa à Venda - Mooca, 3 Quartos, 267 m2",3 quartos | 267m² | Condomínio: R$ 0 | 2...,1350000.0,https://img.olx.com.br/thumbs256x256/41/416231...,20,Brasil;TO;DDD 63 - Tocantins;Outras Cidades,"Araguaína, Loteamento São Francisco",1,2022-06-09 08:19,2022-

In [77]:
def texto_para_asc2(text:str)->str: #Remove acentos e deixa toda a string minuscula
    frase = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower().strip()
    return re.sub("[^\w]+"," ",frase)#re.sub("\,|\.|\-|\[|\]|\(|\)|\'|\!","",frase) 


artigos_preposicoes = stopwords.words('portuguese')
acentos = ['.',',','(',')','[',']',"'"]

In [85]:
dicionario_categoria_palavras = {}

In [84]:
def add_categoria_dicionario(vet_titulos, categoria,dicionario):
    if not dicionario.get(categoria):
        dicionario[categoria] = {}
    for titulo in vet_titulos: 
        palavras_titulo = texto_para_asc2(titulo).split(" ")
        for palavra in palavras_titulo: 
            _palavra = palavra.strip()
            if _palavra not in artigos_preposicoes and len(_palavra) > 2 and not re.match("(\w|d)*[\d](\w|d)*",_palavra):
                if dicionario[categoria].get(_palavra):
                    dicionario[categoria][_palavra] += 1
                else: 
                    dicionario[categoria][_palavra] = 1

In [72]:
vet_teste = ['livro de matemática','casa do joão pedro','Teste Ana Clara Casa']
add_categoria_dicionario(vet_teste, 'coisas aleatorias', dicionario_categoria_palavras)

In [86]:
dataframe_categoria1 = pd.read_sql(monta_sql_por_categoria(lista_todas_categorias[0]),con)
titulos_categoria1 = dataframe_categoria1['titulo_anuncio'].to_list()

In [87]:
add_categoria_dicionario(titulos_categoria1, lista_todas_categorias[0],dicionario_categoria_palavras)


In [89]:
len(dicionario_categoria_palavras[lista_todas_categorias[0]].keys())

37846

In [93]:
conta = 0
for chave in dicionario_categoria_palavras[lista_todas_categorias[0]].keys():
    if dicionario_categoria_palavras[lista_todas_categorias[0]][chave] > 5:
        conta += 1
    else: print(chave)
print(conta)

uninorte
portale
hevia
quadro
parkia
bollevard
frigobar
aptm
ramage
shopingpatio
gior
kaa
pacoal
quartossuite
placic
dentr
raffaello
sanzio
saron
vilaris
produban
verdeed
palazzi
orizzonte
mahatma
ghandi
versatile
lares
delmare
divane
vcbeira
mis
gbar
perfei
taylor
made
nubia
antenas
unicompra
perreli
race
lazulli
dior
galba
accioly
summit
torrar
difer
dhiana
galda
acoca
granelle
maira
sirah
termincao
manga
coc
bares
restaurant
colil
pensad
scarp
gulandi
lev
anthony
mobilidado
paixao
perfe
status
tourlon
invsista
bodega
lopana
completi
najatiuca
patmos
ananda
ledo
vuitton
mellilas
raffaelo
houston
passsaros
edidicio
ptivilege
clas
ethos
milones
porteria
lavoisier
terminacao
tabriz
xtra
pratagi
acompanhar
beli
dpe
jat
verdite
merom
comporto
noedf
seychelles
entremares
delmam
firmino
atraente
cheiro
louge
pierrakos
pverde
hidromas
grandioso
marcel
greenvillage
naturepark
orizzont
gatto
autonoma
jatiuc
leiria
aaprtamento
pinot
blumare
educar
nadyr
casuarinas
terceiros
tuoan
miliones
stel
